# Enviroweather Private Weather Stations Python package: `ewx_pws`
## Example usage

Using this package in a project to pull weather data you start as follows

In [1]:
import json
from pprint import pprint
from ewx_pws import __version__, ewx_pws
print(__version__)

0.1.0


This package works with the following station types:

In [3]:
from weather_stations import STATION_TYPE_LIST
print(STATION_TYPE_LIST)

['ZENTRA', 'ONSET', 'DAVIS', 'RAINWISE', 'SPECTRUM', 'LOCOMOS', 'GENERIC']


The main function accepts configuration information for one or more weather stations API (clouds), 
and uses that to connect to the API for that station and pull data for a specific time interval. 

The configuration can be saved in a CSV file or in the future in a database table. 

This example usage notebook assumes the values are in a CSV file named 'test_stations.csv'  Alter the variable for this file name
to use a different file. 

The file must be in the format :  


In [4]:
# this file must be present for the rest of the notebook to work.  You can create this file with any name, 
# but must be loaded here
station_config_file = "../test_stations.csv"

In [5]:
# load weather station configuration from an environment file
stations = ewx_pws.station_dict_from_file(station_config_file)

print("full station config:")
for station in stations.values():
    print(station.config)

2023-12-04 23:17:01,761-9444-DEBUG-instantiating EWX-METER-FO
2023-12-04 23:17:01,762-9444-DEBUG-instantiating EWXDAVIS01
2023-12-04 23:17:01,762-9444-DEBUG-instantiating EWXSPECTRUM01
2023-12-04 23:17:01,762-9444-DEBUG-instantiating EWXONSET01
2023-12-04 23:17:01,763-9444-DEBUG-instantiating EWXRAINWISE01
2023-12-04 23:17:01,763-9444-DEBUG-instantiating EWX-METER-FO2


full station config:
station_id='EWX-METER-FO' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='ZENTRA' tz='ET' station_config='{"sn":"z6-12564","token":"5b8f637a718b9e3ad6e31eec81f0b51d91ac38bd"}' sn='z6-12564' token='5b8f637a718b9e3ad6e31eec81f0b51d91ac38bd'
station_id='EWXDAVIS01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='DAVIS' tz='ET' station_config='{"sn":"117005","apikey":"aoqdcbirudd1sarq6erfj6tgtw67sbvo","apisec":"elvr77yhdx3p0286cwnnqqstdwqg8zxf"}' sn='117005' apikey='aoqdcbirudd1sarq6erfj6tgtw67sbvo' apisec='elvr77yhdx3p0286cwnnqqstdwqg8zxf'
station_id='EWXSPECTRUM01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='SPECTRUM' tz='ET' station_config='{"sn":"50400123","apikey":"11a5c3a939856b08677b7a072f8e6865"}' sn='50400123' apikey='11a5c3a939856b08677b7a072f8e6865'
station_id='EWXONSET01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='ONSET' tz='ET' station_config='{"sn":"21092695","client_id":"Enviroweather_W

In [6]:
print("stations in config file:")
for station in stations.values():
    print('----------')
    print('id:      ', station.id)
    print('type:    ', station.config.station_type)
    print('timezone:', station.config.tz)
    print('sample interval:', station.interval_min)



stations in config file:
----------
id:       EWX-METER-FO
type:     ZENTRA
timezone: ET
sample interval: 5
----------
id:       EWXDAVIS01
type:     DAVIS
timezone: ET
sample interval: 15
----------
id:       EWXSPECTRUM01
type:     SPECTRUM
timezone: ET
sample interval: 5
----------
id:       EWXONSET01
type:     ONSET
timezone: ET
sample interval: 5
----------
id:       EWXRAINWISE01
type:     RAINWISE
timezone: ET
sample interval: 15
----------
id:       EWX-METER-FO2
type:     ZENTRA
timezone: ET
sample interval: 5


Get sample weather specifying a time interval.,  UTCInterval is a class that can create intervals of recent times, and ensure times created are UTC so The station classes only accept timestamps that have a timezone, and that timezone must be UTC.   

In [7]:
# set the time 
from ewx_pws.time_intervals import UTCInterval
interval = UTCInterval.previous_interval(delta_mins = 60)
interval.dict()

/var/folders/cg/k5g9swk52z941zbwf6c7p2tc0000gn/T/ipykernel_9444/4263527815.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  interval.dict()


{'start': datetime.datetime(2023, 12, 5, 3, 15, tzinfo=datetime.timezone.utc),
 'end': datetime.datetime(2023, 12, 5, 4, 15, tzinfo=datetime.timezone.utc)}

In [8]:
# get weather one station from the list.  test by changing the station_id with one from above
station_id = 'EWX-METER-FO2'
station = stations[station_id]

pprint(station.config)

ZentraConfig(station_id='EWX-METER-FO2', install_date=datetime.datetime(2023, 9, 10, 0, 0), station_type='ZENTRA', tz='ET', station_config='{"sn":"z6-20553","token":"12cc21d9f697e478885cca9b65307d27872dfb18"}', sn='z6-20553', token='12cc21d9f697e478885cca9b65307d27872dfb18')


The output uses a class WeatherReading which has metadata, and then a list of responses from the API.  Some APIs require multiple calls to the API for periods > 24 hours, so the responses are always stored in a list

In [9]:
api_data = station.get_readings(interval.start, interval.end)
pprint(api_data)


2023-12-04 23:17:17,110-9444-DEBUG-Starting new HTTPS connection (1): zentracloud.com:443
2023-12-04 23:17:25,726-9444-DEBUG-https://zentracloud.com:443 "GET /api/v4/get_readings/?device_sn=z6-20553&start_date=2023-12-04+22%3A15%3A00&end_date=2023-12-04+23%3A15%3A00&page_num=1&per_page=1000 HTTP/1.1" 200 None


WeatherAPIData(station_id='EWX-METER-FO2', station_type='ZENTRA', request_id='463df378-e085-4075-a01e-a74e5c35b3fe', request_datetime=datetime.datetime(2023, 12, 5, 9, 17, 16, 892725, tzinfo=datetime.timezone.utc), time_interval=UTCInterval(start=datetime.datetime(2023, 12, 5, 3, 15, tzinfo=datetime.timezone.utc), end=datetime.datetime(2023, 12, 5, 4, 15, tzinfo=datetime.timezone.utc)), package_version='0.1.0', responses=[WeatherAPIResponse(url='https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-12-04+22%3A15%3A00&end_date=2023-12-04+23%3A15%3A00&page_num=1&per_page=1000', status_code='200', reason='OK', text='{"pagination": {"per_page": 1000, "page_num": 1, "next_url": "https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-12-04+22%3A15%3A00&end_date=2023-12-04+23%3A15%3A00&page_num=2&per_page=1000", "max_mrid": 68522, "page_num_readings": 13, "page_num_outputs": 312, "page_start_date": "2023-12-04 22:15:00-05:00", "page_end_date"

## raw api output

let's look at what comes out of the station API for our example station.  The actual data from the api is in the responses property of the weather_data object.  This is an array as some stations return an array of responses (Davis only allows maximum 24hrs per reading)

The folllowing looks at just the first response in the array

In [ ]:
for response in  api_data.responses:
    pprint(response.text)

The output from the api is a mix of the JSON output from the stations and metadata about how the request for data was made.   Each station has it's own format from the API, and we want to transform that to a standardize format for EWX. 

The station object stores the most recent request, and can run the `transform` method without an argument, and it will to use the latest api request, or you can send data. 



In [10]:
api_data.model_dump_record()

{'station_id': 'EWX-METER-FO2',
 'station_type': 'ZENTRA',
 'request_id': '463df378-e085-4075-a01e-a74e5c35b3fe',
 'request_datetime': datetime.datetime(2023, 12, 5, 9, 17, 16, 892725, tzinfo=datetime.timezone.utc),
 'time_interval': UTCInterval(start=datetime.datetime(2023, 12, 5, 3, 15, tzinfo=datetime.timezone.utc), end=datetime.datetime(2023, 12, 5, 4, 15, tzinfo=datetime.timezone.utc)),
 'package_version': '0.1.0',
 'responses': '[{"url":"https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-12-04+22%3A15%3A00&end_date=2023-12-04+23%3A15%3A00&page_num=1&per_page=1000","status_code":"200","reason":"OK","text":"{\\"pagination\\": {\\"per_page\\": 1000, \\"page_num\\": 1, \\"next_url\\": \\"https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-12-04+22%3A15%3A00&end_date=2023-12-04+23%3A15%3A00&page_num=2&per_page=1000\\", \\"max_mrid\\": 68522, \\"page_num_readings\\": 13, \\"page_num_outputs\\": 312, \\"page_start_date\\": \\"2023

In [11]:
readings = station.transform(api_data)
for reading in readings:
    pprint(reading)

2023-12-04 23:17:40,918-9444-DEBUG-Zentra readings found
2023-12-04 23:17:40,919-9444-DEBUG-transformed_reading type <class 'dict_values'>: dict_values([{'data_datetime': datetime.datetime(2023, 12, 5, 4, 15, tzinfo=datetime.timezone.utc), 'atemp': 1.7, 'lws0': 0.0, 'pcpn': 0.0, 'relh': 85.4}, {'data_datetime': datetime.datetime(2023, 12, 5, 4, 10, tzinfo=datetime.timezone.utc), 'atemp': 1.7, 'lws0': 0.0, 'pcpn': 0.0, 'relh': 85.4}, {'data_datetime': datetime.datetime(2023, 12, 5, 4, 5, tzinfo=datetime.timezone.utc), 'atemp': 1.7, 'lws0': 0.0, 'pcpn': 0.0, 'relh': 85.4}, {'data_datetime': datetime.datetime(2023, 12, 5, 4, 0, tzinfo=datetime.timezone.utc), 'atemp': 1.7, 'lws0': 0.0, 'pcpn': 0.0, 'relh': 84.8}, {'data_datetime': datetime.datetime(2023, 12, 5, 3, 55, tzinfo=datetime.timezone.utc), 'atemp': 1.6, 'lws0': 0.0, 'pcpn': 0.0, 'relh': 85.5}, {'data_datetime': datetime.datetime(2023, 12, 5, 3, 50, tzinfo=datetime.timezone.utc), 'atemp': 1.7, 'lws0': 0.0, 'pcpn': 0.0, 'relh': 85.1

{'pagination': {'per_page': 1000, 'page_num': 1, 'next_url': 'https://zentracloud.com/api/v4/get_readings/?device_sn=z6-20553&start_date=2023-12-04+22%3A15%3A00&end_date=2023-12-04+23%3A15%3A00&page_num=2&per_page=1000', 'max_mrid': 68522, 'page_num_readings': 13, 'page_num_outputs': 312, 'page_start_date': '2023-12-04 22:15:00-05:00', 'page_end_date': '2023-12-04 23:15:00-05:00', 'page_start_mrid': 68510, 'page_end_mrid': 68522}, 'data': {'Air Temperature': [{'metadata': {'device_sn': 'z6-20553', 'device_name': 'envwx-meter-fo2', 'port_number': 1, 'sensor_sn': 'ATM-410008286', 'sensor_name': 'ATMOS 41', 'units': ' °C'}, 'readings': [{'timestamp_utc': 1701749700, 'datetime': '2023-12-04 23:15:00-05:00', 'tz_offset': -18000, 'value': 1.7, 'precision': 1, 'mrid': 68522, 'error_flag': False, 'error_description': None}, {'timestamp_utc': 1701749400, 'datetime': '2023-12-04 23:10:00-05:00', 'tz_offset': -18000, 'value': 1.7, 'precision': 1, 'mrid': 68521, 'error_flag': False, 'error_descrip

## Using the data

if Pandas is installed, can read that transformed data into a table as follows

In [12]:
import pandas
from IPython.display import display
weather_df = pandas.DataFrame(readings.model_dump_record())
display(weather_df)

,station_id,station_type,request_id,request_datetime,time_interval,data_datetime,atemp,pcpn,relh,lws0
0,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 04:15:00+00:00,1.7,0.0,85.4,0.0
1,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 04:10:00+00:00,1.7,0.0,85.4,0.0
2,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 04:05:00+00:00,1.7,0.0,85.4,0.0
3,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 04:00:00+00:00,1.7,0.0,84.8,0.0
4,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 03:55:00+00:00,1.6,0.0,85.5,0.0
5,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 03:50:00+00:00,1.7,0.0,85.1,0.0
6,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 03:45:00+00:00,1.6,0.0,85.5,0.0
7,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 03:40:00+00:00,1.7,0.0,84.7,0.0
8,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 03:35:00+00:00,1.7,0.0,84.5,0.0
9,EWX-METER-FO2,ZENTRA,463df378-e085-4075-a01e-a74e5c35b3fe,2023-12-05 09:17:16.892725+00:00,"{'start': 2023-12-05 03:15:00+00:00, 'end': 20...",2023-12-05 03:30:00+00:00,1.8,0.0,83.5,0.0
